In [8]:
#Libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader import data as wb
import scipy.optimize as sco

In [9]:
tickers_list = ['CRM', 'NVDA', 'ADBE', 'GS', 'PG', 'SMCI', 'NEE']

Adjclose = yf.download(tickers_list,period = '2y')['Adj Close']

[*********************100%%**********************]  7 of 7 completed


In [10]:
pctchangeclose=Adjclose.pct_change()
Table1 = pd.DataFrame(index=tickers_list)
Table1['Variance']=(pctchangeclose*100).var()
Table1['Mean']=pctchangeclose.mean()
Table1

,Variance,Mean
CRM,6.417451,-0.000497
NVDA,13.772685,0.001588
ADBE,6.521060,-0.000038
GS,2.874024,-0.000379
PG,1.389421,0.000269
SMCI,19.509178,0.004687
NEE,3.334313,-0.000507


In [11]:
correlation = (pctchangeclose*100).corr()
correlation

,ADBE,CRM,GS,NEE,NVDA,PG,SMCI
ADBE,1.000000,0.689923,0.421129,0.334395,0.661122,0.249731,0.334583
CRM,0.689923,1.000000,0.454418,0.323376,0.599571,0.214512,0.333964
GS,0.421129,0.454418,1.000000,0.308662,0.452905,0.288021,0.308772
NEE,0.334395,0.323376,0.308662,1.000000,0.245674,0.427488,0.090225
NVDA,0.661122,0.599571,0.452905,0.245674,1.000000,0.164683,0.487831
PG,0.249731,0.214512,0.288021,0.427488,0.164683,1.000000,0.089421
SMCI,0.334583,0.333964,0.308772,0.090225,0.487831,0.089421,1.000000


In [12]:
#https://medium.com/@zeng.simonl/the-efficient-frontier-in-python-a1bc9496a0a1
#Pages 424 - 425 Python for Finance 
def port_ret(weights):
    return np.sum(pctchangeclose.mean() * weights) * 252

def port_vol(weights):
    return np.sqrt(np.dot(weights.T, np.dot(pctchangeclose.cov() * 252, weights)))



In [ ]:
retCol = []
volCol = []
wei =[]

for p in range (400000):
    
    weights = np.random.random(len(tickers_list))
    weights /= np.sum(weights)
    
    wei.append(weights)
    
    retCol.append(port_ret(weights))
    volCol.append(port_vol(weights))


ER = np.array(retCol)
EV = np.array(volCol)


sharpe=(ER-0.02)/EV

In [ ]:
plt.figure(figsize=(10,6 ))
plt.scatter(EV, ER, c=sharpe,
marker='.', cmap='coolwarm')
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.colorbar(label='Sharpe ratio');

In [ ]:
tbl2=pd.DataFrame(columns=['Expected Return',
                           'Volatility',
                           'Optimal Sharpe Ratio'])
tbl2['Expected Return']=ER
tbl2['Volatility']=EV
tbl2['Optimal Sharpe Ratio']=sharpe

for counter, symbol in enumerate(pctchangeclose.columns.tolist()):
    tbl2[symbol + "'s weight"] = [w[counter] for w in wei]


tbl2.set_index("Optimal Sharpe Ratio", inplace=True)
portfolios = pd.DataFrame(tbl2)
tbl2F=portfolios.head().T
tbl2F

In [ ]:
def sharpeMinzd(w):
    return -(port_ret(w)-.02) / port_vol(w)

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for x in range(len(tickers_list)))
eWV = np.array(len(tickers_list) * [1. / len(tickers_list),])

sharpeMinzd(eWV)             

In [ ]:
# Sharpe Ratio Maximization Optimization
sharpeOptimized = sco.minimize(
    fun=sharpeMinzd,        # Objective function (negative Sharpe ratio)
    x0=eWV,                # Initial portfolio weights
    method='SLSQP',        # Optimization method
    bounds=bounds,         # Portfolio weight bounds
    constraints=constraints  # Optimization constraints
)

# Minimum Volatility Portfolio Optimization
volOptimized = sco.minimize(
    fun=port_vol,           # Objective function (portfolio volatility minimization)
    x0=eWV,                # Initial portfolio weights
    method='SLSQP',        # Optimization method
    bounds=bounds,         # Portfolio weight bounds
    constraints=constraints  # Optimization constraints
)


In [ ]:
cons = ({'type': 'eq', 'fun': lambda x: port_ret(x) - treturn},
               {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bnds = tuple((0, 1) for x in weights)

treturns = np.linspace(0.05, 0.2, 50)
tvolatility = []

for treturn in treturns:
    res = sco.minimize(port_vol, eWV, method='SLSQP',
                       bounds=bnds, 
                       constraints=cons)
    tvolatility.append(res['fun'])
tvolatility = np.array(tvolatility)

In [ ]:
#Optimal Portfolio Returns
OPR=port_ret(sharpeOptimized['x'])
OPR.round(4)
0.156
#Optimal Portfolio Volatility Weighting
OPVW=port_vol(sharpeOptimized['x'])
OPVW.round(4)
0.1679
#Sharpe Ratio Maximized
SRM=(OPR-0.02)/OPVW
SRM.round(4)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(EV, ER, c=sharpe,marker='.', alpha=0.8, cmap='coolwarm')
plt.plot(tvolatility, treturns, 'b', lw=4.0)
plt.plot(port_vol(sharpeOptimized['x']), port_ret(sharpeOptimized['x']),
         'y*', markersize=15.0)
plt.plot(port_vol(volOptimized['x']), port_ret(volOptimized['x']),
         'r*', markersize=15.0)
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.colorbar(label='Sharpe ratio')